# Intersight Python SDK Query Examples

The following examples use Intersight's query languange to get information from several Intersight API resources.  If you haven't already installed the Python SDK, please follow the detailed instructions in the [operations notebook](https://github.com/CiscoDevNet/intersight-jupyter-notebooks/blob/main/intersight_ucs_x_operations_python_sdk.ipynb).

To get started, we'll create an API client and use it to connect with Intersight.

> The values below only serve as an example so they won't work in your environment.  Be sure to enter the values for your account in the `key_id` variable below as well as the `private_key_path`.  Also be sure that you are using v3 API keys from your Intersight account.

In [ ]:
import intersight

configuration = intersight.Configuration(
    signing_info=intersight.HttpSigningConfiguration(
        key_id='596cc79e5d91b400010d15ad/5db71f977564612d30cc3860/5e9217a57564612d302f475b',
        private_key_path='/Users/dsoper/Downloads/SecretKey.txt',
        signing_scheme=intersight.signing.SCHEME_HS2019,
        signed_headers=[intersight.signing.HEADER_HOST,
                        intersight.signing.HEADER_DATE,
                        intersight.signing.HEADER_DIGEST,
                        intersight.signing.HEADER_REQUEST_TARGET
                        ]
    )
)
api_client = intersight.ApiClient(configuration)


## Inventory Query Examples

Now that you are authenticated, let's use the SDK to query server and related resources in Intersight.  You can read more on [Intersight's API query language](https://www.intersight.com/apidocs/introduction/query/), and to get an idea of the resources used by the API you can look at the URIs your browser uses (compute/physical-summaries below).

![Browser Compute Summary](images/browser_compute_phys_summary.png)

The Python example below sets up a query filter to get a count of servers and selects attributes to return.

In [ ]:
import intersight.api.compute_api

# Create physical summary (server) class instance
api_instance = intersight.api.compute_api.ComputeApi(api_client)

# Find the count of servers in the account
server_query = api_instance.get_compute_physical_summary_list(count=True)
print(server_query)

# Intersight limits the number of items returned.  Page through returned results and select Name, Model, Serial
per_page = 50
query_select = "Name,Model,Serial"
for i in range(0, server_query.count, per_page):
    page_query = api_instance.get_compute_physical_summary_list(top=per_page, skip=i, select=query_select)
    print(page_query)


### Detailed Inventory

Building on the server inventory example above, we'll now look at Intersight's resource model to get information on server components like physical disks.  We can search in the API for specific disk Model numbers or search for Models that contain a certian substring.  Below we will search for physical disks Model numbers that contain a certain substring. 

In [ ]:
import intersight.api.storage_api

# Create storage class instance
api_instance = intersight.api.storage_api.StorageApi(api_client)

# Setup query options
query_filter = "contains(Model,'UCS')"
query_select = "Dn,Model,RegisteredDevice,RunningFirmware"
query_expand = "RegisteredDevice($select=DeviceHostname),RunningFirmware($select=Version)"

# Get physcial disks that contain the Model substring defined above
storage_query = api_instance.get_storage_physical_disk_list(filter=query_filter, select=query_select, expand=query_expand)
print(storage_query)


### Telemetry Data

Intersight periodically collects data from certain resources, and the API can be used to retrieve the data as a time series.  Below is an example of retrieving time series data.

In [ ]:
from pprint import pformat
from datetime import datetime, timedelta

import intersight.api.telemetry_api
import intersight.model.telemetry_druid_time_series_request
import intersight.model.telemetry_druid_data_source
import intersight.model.telemetry_druid_period_granularity
import intersight.model.telemetry_druid_aggregator

# Create an instance of the telemtry service
api_instance = intersight.api.telemetry_api.TelemetryApi(api_client)

# Build the request model for ethernet port statistics over a given time period
end_time = datetime.now()
start_time = end_time - timedelta(days=2)
# Convert date object to formatted date string for use with intersight queries
start_str = f"{start_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]}Z"
end_str = f"{end_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]}Z"
req_model = intersight.model.telemetry_druid_time_series_request.TelemetryDruidTimeSeriesRequest(
    query_type="timeseries",
    data_source=intersight.model.telemetry_druid_data_source.TelemetryDruidDataSource(
        type="table",
        name="ucs_ether_port_stat",
    ),
    intervals=[
        f"{start_str}/{end_str}"
    ],
    granularity=intersight.model.telemetry_druid_period_granularity.TelemetryDruidPeriodGranularity(
        type="period",
        period="PT12H",
    ),
    aggregations=[
        intersight.model.telemetry_druid_aggregator.TelemetryDruidAggregator(
            field_name="sumBytesTx",
            type="longSum",
            name="traffic",
            field_names=["sumBytesTx"]
        ),
    ]
)

# Query the telemetry API using the request model
telemetry_data = api_instance.query_telemetry_time_series(telemetry_druid_time_series_request=req_model)
print(pformat(telemetry_data))


## Additional Examples

Several additional examples of using the SDK are on GitHub in the [Intersight Python Utilties repo](https://github.com/CiscoDevNet/intersight-python-utils).  The repo also contains a credentials.py module to simplify authentication across all of the example scripts.